<a href="https://colab.research.google.com/github/yiboxu20/MachineLearning/blob/main/Resources/Module1/KNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pylab inline
from IPython.display import Image
import numpy.linalg as LA

Populating the interactive namespace from numpy and matplotlib


## Curse of Dimensionality
The main statistical problem with KNN classifiers is that they do not work well with high dimensional
inputs, due to the **curse of dimensionality**.

The basic problem is that the volume of space grows exponentially fast with dimension, so you
might have to look quite far away in space to find your nearest neighbor.

Also in our algorithm, each time we need to exhaust all training points to find the $k$-nearest neighbours. It is very expensive and inefficient. (this can be solved by using some anchor points and other various hashing methods.)

So it is important to use a metric that only cares about a subset of the dimensions.

## Large margin nearest neighbor

Learn a distance function (pseudo-metric) designed for $k$-NN of the type, **Mahalanobis metric**:
$$d_M(\mathbf{x},\mathbf{z})= \sqrt{(\mathbf{x}-\mathbf{z})^\top M (\mathbf{x}-\mathbf{z})},$$
 where $M$ is a PSD. Particularly, $\mathrm{rank}(M)$ may be less than $d$.

 The goal of the algorithm is to distinguish between two types of special data points:
 - **target neighbors** of $\mathbf{x}^{(i)}$: $k$ different neighbors from the same class as $\mathbf{x}^{(i)}$, selected before learning. Target neighbors are the samples that should become nearest neighbors under the learned Mahalanobis metric.

 - **impostors** of $\mathbf{x}^{(i)}$: neighbor samples $\mathbf{x}^{\ell}$ with $\ell\in \mathcal{N}_k(\mathbf{x}^{(i)})$ from a different class, i.e, $y^{(\ell)}\ne y^{(i)}$.


In [2]:
Image(url='https://github.com/yiboxu20/MachineLearning/blob/main/Resources/images/Lmnn.png?raw=true', width=700)

- The first optimization goal is achieved by minimizing the average distance between each data point $\mathbf{x}^{(i)}$ and their target neighbors, i.e.
 $$\sum_{i=1}^N \sum_{j\in \mathcal{N}_k(\mathbf{x}^{(i)})}  d_M^2(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})=\sum_{i=1}^N\sum_{j\in \mathcal{N}_k(\mathbf{x}^{(i)}) }(\mathbf{x}^{(i)}-\mathbf{x}^{(j)})^\top M (\mathbf{x}^{(i)}-\mathbf{x}^{(j)}) $$

- The second optimization goal to make sure the impostors should be far away. Distance to impostors $\mathbf{x}^{(\ell)}$ are more than 1 unit further away than to target neighbors $\mathbf{x}^{(j)}$:

  $$d^2_M(\mathbf{x}^{(i)} , \mathbf{x}^{(\ell)})-d^2_M(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})\ge 1, \forall i, j,\ell\in \mathcal{N}_k(\mathbf{x}^{(i)}), \ell: y^{(\ell)}\ne y^{(i)}$$
   The margin of exactly one unit fixes the scale of the matrix $M$. This is hard margin constraint.


### Soft Margin
Introduce the slack variable $\xi$ to allow violations of margin constraint, with $\xi_{ij\ell}\ge0$
 $$d^2_M(\mathbf{x}^{(i)} , \mathbf{x}^{(\ell)})-d^2_M(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})\ge 1-\xi_{ij\ell}, \forall i, j,\ell\in \mathcal{N}_k(\mathbf{x}^{(i)}), \ell: y^{(\ell)}\ne y^{(i)}$$

Use the hinge loss function to rewrite the optimization, and solve the following semidefinite programming (SDP):
$$\min_{M\in \mathbb{R}^{d\times d}, \xi} \sum_{i=1}^N \sum_{j\in \mathcal{N}_k(\mathbf{x}^{(i)})}d_M^2(\mathbf{x}^{(i)}, \mathbf{x}^{(j)}) + \lambda \sum_{i=1}^N \sum_{j\in \mathcal{N}_k(\mathbf{x}^{(i)})} \sum_{\ell: y^{(\ell)}\ne y^{(i)}} \max\{0, d^2_M(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})- d^2_M(\mathbf{x}^{(i)} ,\mathbf{x}^{(\ell)}) + 1 \}$$
subject to $M\ge 0$

- $\lambda>0$ is again the regularization parameter.

- Similarly, the slack variable $\xi_{ij\ell}\ge 0$ measures the amount by which the margin inequality is violated.

- However, it is more difficult optimization problem since it has an extra PSD constraint.

- often perform dimensional reduction on data to make $d$ smaller and thus
to speed up learning.

This problem can be solved by `scipy` package `minimize` function with `L-BFGS-B` method.


This demo code is in [KNN2_code](https://github.com/yiboxu20/MachineLearning/blob/main/Resources/Module1/KNN2_code.ipynb).  This is mainly from https://github.com/johny-c/pylmnn.

## Summary
- Non-parameterized algorithm for classification (majority vote) and
regression (averaging).

- Simple and intuitive: require no explicit training step or optimization
process.

- Need storage of training data at inference time; memory intensive for
large-scale datasets.

- Computationally expensive: $O(Nd)$ computations for search through all
training data at inference time. Some smart methods are possible to significantly decrease the computations.

- A proper distance function can be learned via metric learning for a given
dataset.
